# Converting from Tensorflow to ONNX
    we will convert ssd resnet50 to onnx and apply the onnx model on image. 

In [27]:
import os
import sys
# change working directory
ROOT = os.path.dirname(os.getcwd())

# load GPU into environnement
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['ROOT'] = ROOT

model = "pre_trained_models/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/saved_model"
## Set needed paths
os.environ['MODEL'] = model
## onnx path
onnx_name = model.split('/')[1]
os.environ['ONNX'] = os.path.join(model.split('/')[0],model.split('/')[1],model.split('/')[1])
##  set onnx opset. ssd and mask suport 11,12,..
os.environ['OPSET'] = '11'
!cd $ROOT

## Check the input and ouptup of the model

In [28]:
!saved_model_cli show --dir $ROOT/$MODEL  --tag_set serve --signature_def serving_default

2021-01-28 16:21:00.067291: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
The given SavedModel SignatureDef contains the following input(s):
  inputs['input_tensor'] tensor_info:
      dtype: DT_UINT8
      shape: (1, -1, -1, 3)
      name: serving_default_input_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['anchors'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 4)
      name: StatefulPartitionedCall:0
  outputs['box_classifier_features'] tensor_info:
      dtype: DT_FLOAT
      shape: (300, 9, 9, 1536)
      name: StatefulPartitionedCall:1
  outputs['class_predictions_with_background'] tensor_info:
      dtype: DT_FLOAT
      shape: (300, 91)
      name: StatefulPartitionedCall:2
  outputs['detection_anchor_indices'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: StatefulPartitionedCall:3
  outputs['detection_boxes'] tensor_info:
      dtype: D

## Convert to onnx using saved_model and a frozen model
    We don't a frozen Model because tensorflow 2.x no longer not provide an convertor to freeze the model
    the supported opset for ssd and mask are 11,12,..

In [29]:
!python -m tf2onnx.convert --opset $OPSET --fold_const --saved-model $ROOT/$MODEL --output $ROOT/$ONNX.onnx

2021-01-28 16:22:56.136642: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-01-28 16:22:58,067 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-01-28 16:23:00.099932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-28 16:23:00.175658: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-01-28 16:23:00.175714: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-01-28 16:23:00.243550: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-01-28 16:23:00.243730: I tensorflow/st

2021-01-28 16:24:43.574329: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:933] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 8836 nodes (-2230), 10776 edges (-2700), time = 5427.9209ms.
  function_optimizer: function_optimizer did nothing. time = 41.024ms.
  constant_folding: Graph size after: 8836 nodes (0), 10776 edges (0), time = 692.448ms.
  function_optimizer: function_optimizer did nothing. time = 12.299ms.

2021-01-28 16:25:01.696218: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-01-28 16:25:01.726275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1902] Adding visible gpu devices: 0
2021-01-28 16:25:01.744614: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1300] Device interconnect StreamExecutor 

2021-01-28 16:25:18.017697: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-01-28 16:25:18.018696: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1902] Adding visible gpu devices: 0
2021-01-28 16:25:18.018752: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1300] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-01-28 16:25:18.018771: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1306]      0 
2021-01-28 16:25:18.018791: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1319] 0:   N 
2021-01-28 16:25:18.019973: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1446] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 4991 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, com

2021-01-28 16:25:18.463355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-01-28 16:25:18.464405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1902] Adding visible gpu devices: 0
2021-01-28 16:25:18.464451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1300] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-01-28 16:25:18.464471: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1306]      0 
2021-01-28 16:25:18.464490: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1319] 0:   N 
2021-01-28 16:25:18.465448: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1446] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 4991 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, com

2021-01-28 16:25:54,070 - WARNING - Node StatefulPartitionedCall/functional_1/freezable_batch_norm_29/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:54,072 - WARNING - Node StatefulPartitionedCall/functional_1/freezable_batch_norm_25/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:54,074 - WARNING - Node StatefulPartitionedCall/functional_1/freezable_batch_norm_26/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:54,076 - WARNING - Node StatefulPartitionedCall/functional_1/freezable_batch_norm_24/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.


...

2021-01-28 16:25:56,146 - WARNING - ONNX Failed to infer shapes and dtypes for [StatefulPartitionedCall/CropAndResize/CropAndResize, type: Loop]
Traceback (most recent call last):
  File "/home/kamgo/.virtualenvs/tfTrainOdIs/lib/python3.8/site-packages/tf2onnx/schemas.py", line 157, in infer_onnx_shape_dtype
    inferred_model = shape_inference.infer_shapes(model_proto)
  File "/home/kamgo/.virtualenvs/tfTrainOdIs/lib/python3.8/site-packages/onnx/shape_inference.py", line 35, in infer_shapes
    inferred_model_str = C.infer_shapes(model_str, check_type)
RuntimeError: Shape inference error(s): (op_type:Slice, node name: Slice_c): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Reshape, node name: Reshape__1049): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Slice, node name: Slice__1055): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Slice, node name: Slice__1052): [TypeInferenceError] In

2021-01-28 16:25:56,436 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_183/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:56,461 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_188/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:56,465 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_189/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:56,469 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_190/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to fal

2021-01-28 16:25:57,049 - WARNING - ONNX Failed to infer shapes and dtypes for [StatefulPartitionedCall/BatchMultiClassNonMaxSuppression_1/MultiClassNonMaxSuppression/unstack__2576, type: Squeeze]
Traceback (most recent call last):
  File "/home/kamgo/.virtualenvs/tfTrainOdIs/lib/python3.8/site-packages/tf2onnx/schemas.py", line 157, in infer_onnx_shape_dtype
    inferred_model = shape_inference.infer_shapes(model_proto)
  File "/home/kamgo/.virtualenvs/tfTrainOdIs/lib/python3.8/site-packages/onnx/shape_inference.py", line 35, in infer_shapes
    inferred_model_str = C.infer_shapes(model_str, check_type)
RuntimeError: Shape inference error(s): (op_type:Squeeze, node name: StatefulPartitionedCall/BatchMultiClassNonMaxSuppression_1/MultiClassNonMaxSuppression/unstack__2576): [ShapeInferenceError] Dimension of input 1 must be 1 instead of 4

2021-01-28 16:25:57,050 - WARNING - ONNX Failed to infer shapes and dtypes for [StatefulPartitionedCall/BatchMultiClassNonMaxSuppression_1/MultiClass

2021-01-28 16:25:59,583 - WARNING - ONNX Failed to infer shapes and dtypes for [StatefulPartitionedCall/CropAndResize_1/CropAndResize, type: Loop]
Traceback (most recent call last):
  File "/home/kamgo/.virtualenvs/tfTrainOdIs/lib/python3.8/site-packages/tf2onnx/schemas.py", line 157, in infer_onnx_shape_dtype
    inferred_model = shape_inference.infer_shapes(model_proto)
  File "/home/kamgo/.virtualenvs/tfTrainOdIs/lib/python3.8/site-packages/onnx/shape_inference.py", line 35, in infer_shapes
    inferred_model_str = C.infer_shapes(model_str, check_type)
RuntimeError: Shape inference error(s): (op_type:Slice, node name: Slice_c): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Reshape, node name: Reshape__8331): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Slice, node name: Slice__8337): [TypeInferenceError] Input 0 expected to have type but instead is null
(op_type:Slice, node name: Slice__8334): [TypeInferenceError] 

2021-01-28 16:25:59,834 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_179/FusedBatchNormV3_1 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:59,859 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_184/FusedBatchNormV3_1 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:59,862 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_185/FusedBatchNormV3_1 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-01-28 16:25:59,866 - WARNING - Node StatefulPartitionedCall/functional_3/freezable_batch_norm_186/FusedBatchNormV3_1 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training se